In [23]:
import numpy as np
from w2v_utils import *

In [26]:
def read_glove(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}

        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)

    return words, word_to_vec_map

Load the word vectors. We will use 50-dimensional GloVe vectors to represent words.

In [27]:
words, word_to_vec_map = read_glove('data\\glove.6B.50d.txt')

- `words`: set of words in the vocabulary.
- `word_to_vec_map`: dictionary mapping words to their GloVe vector representation.

## Cosine Similarity

To measure how similar two words are, we need a way to measure the degree of similarity between two embedding vectors for the two words. Given two vectors $u$ and $v$, cosine similarity is defined as follows: 

$$\text{CosineSimilarity(u, v)} = \frac {u . v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}$$

where $u.v$ is the dot product (or inner product) of two vectors, $||u||_2$ is the norm (or length) of the vector $u$, and $\theta$ is the angle between $u$ and $v$. This similarity depends on the angle between $u$ and $v$. If $u$ and $v$ are very similar, their cosine similarity will be close to 1; if they are dissimilar, the cosine similarity will take a smaller value.

In [28]:
def cosine_similarity(u, v):
    distance = 0.0
    
    dot = np.dot(u,v)
    
    norm_u = np.sqrt(np.sum(np.square(u)))

    norm_v = np.sqrt(np.sum(np.square(v)))

    cosine_similarity = dot / (norm_u * norm_v)

    return cosine_similarity

In [30]:
father = word_to_vec_map["father"]
mother = word_to_vec_map["mother"]
ball = word_to_vec_map["ball"]
crocodile = word_to_vec_map["crocodile"]
france = word_to_vec_map["france"]
italy = word_to_vec_map["italy"]
paris = word_to_vec_map["paris"]
rome = word_to_vec_map["rome"]

print("cosine_similarity(father, mother) = ", cosine_similarity(father, mother))
print("cosine_similarity(ball, crocodile) = ",cosine_similarity(ball, crocodile))
print("cosine_similarity(france - paris, rome - italy) = ",cosine_similarity(france - paris, rome - italy))
print("cosine_similarity(france - paris, italy - rome) = ",cosine_similarity(france - paris, italy - rome))

cosine_similarity(father, mother) =  0.8909038442893615
cosine_similarity(ball, crocodile) =  0.2743924626137942
cosine_similarity(france - paris, rome - italy) =  -0.6751479308174201
cosine_similarity(france - paris, italy - rome) =  0.6751479308174201


## Word Analogy Task

In the word analogy task, we complete the sentence <font color='brown'>"*a* is to *b* as *c* is to **____**"</font>. An example is <font color='brown'> '*man* is to *woman* as *king* is to *queen*' </font>. In detail, we are trying to find a word *d*, such that the associated word vectors $e_a, e_b, e_c, e_d$ are related in the following manner: $e_b - e_a \approx e_d - e_c$. We will measure the similarity between $e_b - e_a$ and $e_d - e_c$ using cosine similarity.

In [31]:
def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
    
    e_a, e_b, e_c = word_to_vec_map[word_a], word_to_vec_map[word_b], word_to_vec_map[word_c]
    
    words = word_to_vec_map.keys()
    max_cosine_sim = -100              
    best_word = None                   

    for w in words:        
        if w in [word_a, word_b, word_c] :
            continue
        
        cosine_sim = cosine_similarity(e_b - e_a, word_to_vec_map[w] - e_c)
        
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = w
        
    return best_word

In [32]:
triads_to_try = [('italy', 'italian', 'spain'), ('india', 'delhi', 'japan'), ('man', 'woman', 'boy'), ('small', 'smaller', 'large')]
for triad in triads_to_try:
    print ('{} -> {} :: {} -> {}'.format( *triad, complete_analogy(*triad,word_to_vec_map)))

italy -> italian :: spain -> spanish
india -> delhi :: japan -> tokyo
man -> woman :: boy -> girl
small -> smaller :: large -> larger


## Debiasing word vectors

We will examine gender biases that can be reflected in a word embedding, and explore algorithms for reducing the bias.
We'll first see how the GloVe word embeddings relate to gender. First compute a vector $g = e_{woman}-e_{man}$, where $e_{woman}$ represents the word vector corresponding to the word *woman*, and $e_{man}$ corresponds to the word vector corresponding to the word *man*. The resulting vector $g$ roughly encodes the concept of "gender".

In [33]:
g = word_to_vec_map['woman'] - word_to_vec_map['man']
print(g)

[-0.087144    0.2182     -0.40986    -0.03922    -0.1032      0.94165
 -0.06042     0.32988     0.46144    -0.35962     0.31102    -0.86824
  0.96006     0.01073     0.24337     0.08193    -1.02722    -0.21122
  0.695044   -0.00222     0.29106     0.5053     -0.099454    0.40445
  0.30181     0.1355     -0.0606     -0.07131    -0.19245    -0.06115
 -0.3204      0.07165    -0.13337    -0.25068714 -0.14293    -0.224957
 -0.149       0.048882    0.12191    -0.27362    -0.165476   -0.20426
  0.54376    -0.271425   -0.10245    -0.32108     0.2516     -0.33455
 -0.04371     0.01258   ]


In [34]:
print ('List of names and their similarities with constructed vector:')

# girls and boys name
name_list = ['john', 'marie', 'sophie', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 'katy', 'yasmin']

for w in name_list:
    print (w, cosine_similarity(word_to_vec_map[w], g))

List of names and their similarities with constructed vector:
john -0.23163356145973724
marie 0.315597935396073
sophie 0.31868789859418784
ronaldo -0.31244796850329437
priya 0.17632041839009405
rahul -0.16915471039231722
danielle 0.24393299216283895
reza -0.07930429672199553
katy 0.28310686595726153
yasmin 0.23313857767928758


As you can see, female first names tend to have a positive cosine similarity with our constructed vector $g$, while male first names tend to have a negative cosine similarity.
But let's try with some other words. 

In [35]:
print('Other words and their similarities:')
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist', 
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']
for w in word_list:
    print (w, cosine_similarity(word_to_vec_map[w], g))

Other words and their similarities:
lipstick 0.27691916256382665
guns -0.1888485567898898
science -0.06082906540929699
arts 0.008189312385880328
literature 0.06472504433459932
warrior -0.20920164641125288
doctor 0.11895289410935041
tree -0.0708939917547809
receptionist 0.3307794175059374
technology -0.13193732447554302
fashion 0.03563894625772698
teacher 0.1792092343182567
engineer -0.08039280494524069
pilot 0.0010764498991916846
computer -0.10330358873850495
singer 0.18500518136496294


It is astonishing how these results reflect certain unhealthy gender stereotypes. For example, "computer" is closer to "man" while "literature" is closer to "woman".

We'll see below how to reduce the bias of these vectors, using an algorithm due to [Boliukbasi et al., 2016](https://arxiv.org/abs/1607.06520). Note that some word pairs such as "actor"/"actress" or "grandmother"/"grandfather" should remain gender specific, while other words such as "receptionist" or "technology" should be neutralized, i.e. not be gender-related. You will have to treat these two type of words differently when debiasing.

## Neutralize bias for non-gender specific words

Removes the bias of "word" by projecting it on the space orthogonal to the bias axis. 

In [36]:
def neutralize(word, g, word_to_vec_map): 
    e = word_to_vec_map[word]
    e_biascomponent =  ( np.dot(e, g) / np.square(np.sqrt((np.sum(np.square(g))))) ) * g
    e_debiased = e - e_biascomponent
    
    return e_debiased

In [37]:
e = "receptionist"
print("cosine similarity between " + e + " and g, before neutralizing: ", cosine_similarity(word_to_vec_map["receptionist"], g))

e_debiased = neutralize("receptionist", g, word_to_vec_map)
print("cosine similarity between " + e + " and g, after neutralizing: ", cosine_similarity(e_debiased, g))

cosine similarity between receptionist and g, before neutralizing:  0.3307794175059374
cosine similarity between receptionist and g, after neutralizing:  -2.099120994400013e-17


## Equalization algorithm for gender-specific words

Let's see how debiasing can also be applied to word pairs such as "actress" and "actor." Equalization is applied to pairs of words that you might want to have differed only through gender property. As a concrete example, suppose that "actress" is closer to "babysit" than "actor." By applying neutralizing to "babysit" we can reduce the gender-stereotype associated with babysitting. But this still does not guarantee that "actor" and "actress" are equidistant from "babysit." The equalization algorithm takes care of this. 

The key idea behind equalization is to make sure that a particular pair of words is equidistant from the 49-dimensional $g_\perp$. The equalization step also ensures that the two equalized steps are now the same distance from $e_{receptionist}^{debiased}$, or from any other work that has been neutralized.

The derivation of the linear algebra to do this is a bit more complex. (See Bolukbasi et al., 2016 for details.) But the key equations are: 

$$ \mu = \frac{e_{w1} + e_{w2}}{2}\tag{1}$$ 

$$ \mu_{B} = \frac {\mu \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{2}$$ 

$$\mu_{\perp} = \mu - \mu_{B} \tag{3}$$

$$ e_{w1B} = \frac {e_{w1} \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{4}$$ 
$$ e_{w2B} = \frac {e_{w2} \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{5}$$


$$e_{w1B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w1B}} - \mu_B} {|(e_{w1} - \mu_{\perp}) - \mu_B)|} \tag{6}$$


$$e_{w2B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w2B}} - \mu_B} {|(e_{w2} - \mu_{\perp}) - \mu_B)|} \tag{7}$$

$$e_1 = e_{w1B}^{corrected} + \mu_{\perp} \tag{8}$$

$$e_2 = e_{w2B}^{corrected} + \mu_{\perp} \tag{9}$$


In [42]:
def equalize(pair, bias_axis, word_to_vec_map):
    
    w1, w2 = pair
    e_w1, e_w2 = word_to_vec_map[w1], word_to_vec_map[w2]
    
    mu = (e_w1 + e_w2) / 2

    mu_B = ((np.dot(mu, bias_axis)) / np.square(np.sqrt((np.sum(np.square(bias_axis)))))) * bias_axis
    mu_orth = mu - mu_B

    e_w1B = ((np.dot(e_w1, bias_axis)) / np.square(np.sqrt((np.sum(np.square(bias_axis)))))) * bias_axis
    e_w2B = ((np.dot(e_w2, bias_axis)) / np.square(np.sqrt((np.sum(np.square(bias_axis)))))) * bias_axis

    corrected_e_w1B = np.sqrt(np.linalg.norm(1 - np.square(np.sqrt((np.sum(np.square(mu_orth))))))) * ((e_w1B - mu_B)/(np.linalg.norm(e_w1 - mu_orth - mu_B)))
    corrected_e_w2B = np.sqrt(np.linalg.norm(1 - np.square(np.sqrt((np.sum(np.square(mu_orth))))))) * ((e_w2B - mu_B)/(np.linalg.norm(e_w2 - mu_orth - mu_B)))

    e1 = corrected_e_w1B + mu_orth
    e2 = corrected_e_w2B + mu_orth
    
    return e1, e2

In [44]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"actor\"], gender) = ", cosine_similarity(word_to_vec_map["actor"], g))
print("cosine_similarity(word_to_vec_map[\"actress\"], gender) = ", cosine_similarity(word_to_vec_map["actress"], g))
print()
e1, e2 = equalize(("actor", "actress"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g))

cosine similarities before equalizing:
cosine_similarity(word_to_vec_map["actor"], gender) =  -0.08387555382505696
cosine_similarity(word_to_vec_map["actress"], gender) =  0.33422494897899774

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  -0.6083328980342392
cosine_similarity(e2, gender) =  0.608332898034239


In [45]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"father\"], gender) = ", cosine_similarity(word_to_vec_map["father"], g))
print("cosine_similarity(word_to_vec_map[\"mother\"], gender) = ", cosine_similarity(word_to_vec_map["mother"], g))
print()
e1, e2 = equalize(("father", "mother"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g))

cosine similarities before equalizing:
cosine_similarity(word_to_vec_map["father"], gender) =  0.03416437478068676
cosine_similarity(word_to_vec_map["mother"], gender) =  0.34016319002265677

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  -0.5516483873227905
cosine_similarity(e2, gender) =  0.5516483873227906


In [47]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"man\"], gender) = ", cosine_similarity(word_to_vec_map["man"], g))
print("cosine_similarity(word_to_vec_map[\"woman\"], gender) = ", cosine_similarity(word_to_vec_map["woman"], g))
print()
e1, e2 = equalize(("man", "woman"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g))

cosine similarities before equalizing:
cosine_similarity(word_to_vec_map["man"], gender) =  -0.11711095765336833
cosine_similarity(word_to_vec_map["woman"], gender) =  0.35666618846270376

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  -0.7004364289309387
cosine_similarity(e2, gender) =  0.7004364289309387


In [49]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"him\"], gender) = ", cosine_similarity(word_to_vec_map["him"], g))
print("cosine_similarity(word_to_vec_map[\"her\"], gender) = ", cosine_similarity(word_to_vec_map["her"], g))
print()
e1, e2 = equalize(("actor", "actress"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g))

cosine similarities before equalizing:
cosine_similarity(word_to_vec_map["him"], gender) =  -0.15880901828164093
cosine_similarity(word_to_vec_map["her"], gender) =  0.2455559107124081

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  -0.6083328980342392
cosine_similarity(e2, gender) =  0.608332898034239


These debiasing algorithms are very helpful for reducing bias, but are not perfect and do not eliminate all traces of bias. For example, one weakness of this implementation was that the bias direction $g$ was defined using only the pair of words _woman_ and _man_.

**References**:
- The debiasing algorithm is from Bolukbasi et al., 2016, [Man is to Computer Programmer as Woman is to
Homemaker? Debiasing Word Embeddings](https://papers.nips.cc/paper/6228-man-is-to-computer-programmer-as-woman-is-to-homemaker-debiasing-word-embeddings.pdf)
- The GloVe word embeddings were due to Jeffrey Pennington, Richard Socher, and Christopher D. Manning. (https://nlp.stanford.edu/projects/glove/)
